In [ ]:

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
dbutils.fs.unmount("/mnt/climate-project")


/mnt/climate-project has been unmounted.


True

In [ ]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com//oauth2/token"}


dbutils.fs.mount(
source = "abfss://climate-project@sejalstorage01.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/climate-project",
extra_configs = configs)

True

In [ ]:
%fs
ls "/mnt/climate-project"

path,name,size,modificationTime
dbfs:/mnt/climate-project/raw-data/,raw-data/,0,1710342647000
dbfs:/mnt/climate-project/transformed-data/,transformed-data/,0,1710342666000


In [ ]:
spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, max, min, corr

# Initialize Spark Session
spark = SparkSession.builder.appName("DataProcessing").getOrCreate()

# Read the data from Azure Blob storage
data_df = spark.read.csv("/mnt/climate-project/raw-data/Data.csv", header=True, inferSchema=True)
country_df = spark.read.csv("/mnt/climate-project/raw-data/Country.csv", header=True, inferSchema=True)
year_df = spark.read.csv("/mnt/climate-project/raw-data/Year.csv", header=True, inferSchema=True)


In [ ]:
data_df.select("`CO2 Emissions (kt)`", "Country", "`Country Code`", "`Population-Total`", "Year").show()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1822414541572145>, line 1
----> 1 data_df.select("`CO2 Emissions (kt)`", "Country", "`Country Code`", "`Population-Total`", "Year").show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3168, in DataFrame.select(self, *cols)
   3123 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   3124     """Projects a set of expressions and returns a new :class:`DataFrame`.
   3125 
   3126     .. versionadded:: 1.3.0
   (...)
   3

In [ ]:
# Displaying the first few rows of the dataset
data_df.show()

# Checking for null values
data_df.select([count(when(col(c).isNull(), c)).alias(c) for c in data_df.columns]).show()

# Dropping rows with null values (if necessary)
data_df = data_df.na.drop()

# Converting data types if required
from pyspark.sql.types import IntegerType
data_df = data_df.withColumn("Year", data_df["Year"].cast(IntegerType()))


+----+------------------+------------+-------------------+-------------------------------------------------+------------------------------------------------+-----------------------------------------------+-----------------------------+----------------------------------------+--------------------------------------+---------------------------------------+-----------------------------------------+-----------------------------------------------------------------+------------------------------------------+----------------------------------------+-----------------+------------------------------------------------------+
|Year|           Country|Country Code|CO2 Emissions (kt) |CO2 Emissions From Gaseous Fuel Consumption (kt) |CO2 Emissions From Liquid Fuel Consumption (kt) |CO2 Emissions From Solid Fuel Consumption (kt) |GDP per Capita (current US$) |Life Expectancy At Birth-Female (years) |Life Expectancy At Birth-Male (years) |Life Expectancy At Birth-Total (years) |Methane Emissions (kt of

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1822414541572146>, line 5
      2 data_df.show()
      4 # Checking for null values
----> 5 data_df.select([count(when(col(c).isNull(), c)).alias(c) for c in data_df.columns]).show()
      7 # Dropping rows with null values (if necessary)
      8 data_df = data_df.na.drop()

File <command-1822414541572146>, line 5, in <listcomp>(.0)
      2 data_df.show()
      4 # Checking for null values
----> 5 data_df.select([count(when(col(c).isNull(), c)).alias(c) for c in data_df.columns]).show()
      7 # Dropping rows with null values (if necessary)
      8 data_df = data_df.na.drop()

NameError: name 'count' is not defined

In [ ]:
from pyspark.sql.functions import sum, avg, col

# Example: Total CO2 Emissions per year
total_CO2_per_year = data_df.groupBy("Year").agg(sum("CO2 Emissions (kt)").alias("Total CO2 Emissions"))

# Example: Average GDP per Capita per Country
avg_gdp_per_country = data_df.groupBy("Country").agg(avg("GDP per Capita (current US$)").alias("Average GDP per Capita"))

# Display the results
total_CO2_per_year.show()
avg_gdp_per_country.show()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1822414541572147>, line 4
      1 from pyspark.sql.functions import sum, avg, col
      3 # Example: Total CO2 Emissions per year
----> 4 total_CO2_per_year = data_df.groupBy("Year").agg(sum("CO2 Emissions (kt)").alias("Total CO2 Emissions"))
      6 # Example: Average GDP per Capita per Country
      7 avg_gdp_per_country = data_df.groupBy("Country").agg(avg("GDP per Capita (current US$)").alias("Average GDP per Capita"))

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pys

In [ ]:
Data.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- CO2 Emissions (kt) : double (nullable = true)
 |-- CO2 Emissions From Gaseous Fuel Consumption (kt) : double (nullable = true)
 |-- CO2 Emissions From Liquid Fuel Consumption (kt) : double (nullable = true)
 |-- CO2 Emissions From Solid Fuel Consumption (kt) : double (nullable = true)
 |-- GDP per Capita (current US$) : double (nullable = true)
 |-- Life Expectancy At Birth-Female (years) : double (nullable = true)
 |-- Life Expectancy At Birth-Male (years) : double (nullable = true)
 |-- Life Expectancy At Birth-Total (years) : double (nullable = true)
 |-- Methane Emissions (kt of CO2 equivalent) : double (nullable = true)
 |-- Nitrous Oxide Emissions (thousand metric tons of CO2 equivalent) : double (nullable = true)
 |-- Population-Female (% of total population) : double (nullable = true)
 |-- Population-Male (% of total population) : double (nullable = 

In [ ]:
Year.printSchema()

root
 |-- Year: integer (nullable = true)



In [ ]:
Country.printSchema()

root
 |-- Country: string (nullable = true)



In [ ]:
# Write the transformed data

Data.coalesce(1).write.option("header", "true").format("com.databricks.spark.csv").save("/mnt/climate-project/transformed/Data.csv")
Country.coalesce(1).write.option("header", "true").format("com.databricks.spark.csv").save("/mnt/climate-project/transformed/Country.csv")

Year.coalesce(1).write.option("header", "true").format("com.databricks.spark.csv").save("/mnt/climate-project/transformed/Year.csv")

# Cleaning up
dbutils.fs.rm("/data", True)


False